# Vaults - Setup

In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from mezo.currency_utils import get_token_price, format_musd_currency_columns
from mezo.datetime_utils import format_datetimes
from mezo.data_utils import add_rolling_values, add_pct_change_columns, add_cumulative_columns
from mezo.clients import BigQueryClient, SubgraphClient, Web3Client

load_dotenv(dotenv_path='../.env', override=True)
COINGECKO_KEY = os.getenv('COINGECKO_KEY')

In [ ]:
# Subgraph Setup

AUGUST_VAULT_SUBGRAPH = "https://api.goldsky.com/api/public/project_cm6ks2x8um4aj01uj8nwg1f6r/subgraphs/mezo-vaults-mezo/1.0.0/gn"

SUBGRAPH_HEADERS = {
        "Content-Type": "application/json",
    }

GET_VAULT_TRANSFERS = """
query vaultTransfers($skip: Int!) {
  transfers(
    first: 1000
    orderBy: timestamp_
    orderDirection: desc
    skip: $skip
) {
    timestamp_
    from
    to
    value
    transactionHash_
    block_number
  }
}
"""

In [ ]:
# Instantiate subgraph

vault = SubgraphClient(
    url = AUGUST_VAULT_SUBGRAPH, 
    headers = SUBGRAPH_HEADERS
)

In [ ]:
# Fetch subgraph data
vaults =  vault.fetch_subgraph_data(GET_VAULT_TRANSFERS, 'transfers')

In [ ]:
# Save vaults data df

vaults_df = pd.DataFrame(vaults)
display(vaults_df)

In [ ]:
vaults_df.columns

In [ ]:
# Exploration 
vaults_df = format_musd_currency_columns(vaults_df, ['value'])

In [ ]:
vaults_df = format_datetimes(vaults_df, ['timestamp_'])

In [ ]:
vaults_df['type'] = 'transfer'  # default value
vaults_df.loc[vaults_df['from'] == '0x0000000000000000000000000000000000000000', 'type'] = 'deposit' 
vaults_df.loc[vaults_df['to'] == '0x0000000000000000000000000000000000000000', 'type'] = 'withdrawal'

# Display the result
display(vaults_df[['from', 'to', 'type']].head(10))
print(f"\nType distribution:")
print(vaults_df['type'].value_counts())

In [ ]:
deposits = vaults_df.loc[vaults_df['from'] == '0x0000000000000000000000000000000000000000']
withdrawals = vaults_df.loc[vaults_df['to'] == '0x0000000000000000000000000000000000000000']

In [ ]:
total_deposit_amt = sum(deposits['value'])
total_withdrawal_amt = sum(withdrawals['value'])
tvl = total_deposit_amt - total_withdrawal_amt

In [ ]:
total_depositors = deposits['to'].nunique()
total_deposits = deposits['to'].count()

In [ ]:
total_withdrawers = withdrawals['from'].nunique()
total_withdrawals = withdrawals['from'].count()

In [ ]:
daily_vault_txns = vaults_df.groupby(['timestamp_']).agg(
    value = ('value', 'sum'),
    transactions = ('count', 'sum'),
    users = ('to', lambda x: x.nunique())
).reset_index().sort_values(by='timestamp_', ascending=False)

In [ ]:
daily_vault_txns

In [ ]:
import requests

vault_contract = "0x221B2D9aD7B994861Af3f4c8A80c86C4aa86Bf53"
url = f"https://api.explorer.mezo.org/api/v2/addresses/{vault_contract}/transactions"
transaction_fees = {}


# Create list to store all transaction data
transactions_list = []
next_page_params = None
page_count = 0
method = 'deposit'

while True:
        page_count += 1
        print(f"Fetching page {page_count}...")

        # Build URL with pagination parameters
        if next_page_params:
                response = requests.get(url, params=next_page_params)
        else:
                response = requests.get(url)

        if response.status_code != 200:
                print(f"Error: {response.status_code} - {response.text}")
                break
                
        data = response.json()
        transactions = data.get("items", [])

        if not transactions:
                print("No more transactions found.")
                break

        print(f"Found {len(transactions)} transactions on page {page_count}")

        for tx in data.get("items", []):
                tx_data = {
                        'timestamp_': tx.get('timestamp'),
                        'method': tx.get('method'),
                        'fee_value': int(tx.get('fee', {}).get('value', 0)) if tx.get('fee') else 0,
                        'has_error': tx.get('has_error_in_internal_txs', False),
                        'from_address': tx.get('from', {}).get('hash') if tx.get('from') else None,
                        'to_address': tx.get('to', {}).get('hash') if tx.get('to') else None,
                        'transactionHash_': tx.get('hash'),
                        'block_number': tx.get('block'),
                }

        # Extract decoded input parameters if available
              # Extract only the first decoded parameter (i=0)
                if tx.get('decoded_input') and tx.get('decoded_input').get('parameters'):
                        parameters = tx.get('decoded_input').get('parameters')
                        if len(parameters) > 0:
                                first_param = parameters[0]
                                tx_data['param_0_name'] = first_param.get('name')
                                tx_data['param_0_value'] = str(first_param.get('value'))
                
                transactions_list.append(tx_data)
          
          # Check for next page
        next_page_params = data.get("next_page_params")
        if not next_page_params:
               print("Reached last page.")
               break

# Create DataFrame
transactions_df = pd.DataFrame(transactions_list)

# Display the DataFrame
print(f"Created DataFrame with {len(transactions_df)} transactions")
print(f"Columns: {list(transactions_df.columns)}")
print(f"Methods found: {transactions_df['method'].value_counts()}")

# Display summary
print(f"\n✅ Created DataFrame with {len(transactions_df)} total transactions")
print(f"📊 Columns: {list(transactions_df.columns)}")
print(f"🔧 Methods found: {transactions_df['method'].value_counts()}")
print(f"📅 Date range: {transactions_df['timestamp_'].min()} to {transactions_df['timestamp_'].max()}")

# Show sample data
print(f"\n🔍 First 5 transactions:")
print(transactions_df.head())

# Show transaction fees by method
print(f"\n💰 Average fees by method:")
fee_by_method = transactions_df.groupby('method')['fee_value'].agg(['count', 'mean','sum']).round(0)
print(fee_by_method)

In [17]:
api_url = f'https://api.explorer.mezo.org/api/v2/addresses/{vault_contract}'
response = requests.get(api_url)
data = response.json()

upMUSD_supply = data.get("token").get('total_supply')
upMUSD_supply =  int(upMUSD_supply) / 1e18
upMUSD_supply

1310238.0359813285